# Mandatory imports and utils

In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Monad
import Control.Monad.Primitive
import Control.Parallel.Strategies

import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial.Chebyshev

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot
:l NB/Heatmap

greyColormap 0.2

# Incomplete beta

Quick reminder about beta function and (regularized) incomplete beta functions:

Beta function:
$$B(a,b) = \int_0^1 t^{a-1}(1 - t)^{b-1} \,dt $$

Incomplete beta:
$$B(x; a,b) = \int_0^x t^{a-1}(1 - t)^{b-1} \,dt \qquad x \in [0,1]$$

Regularized incomplete beta (from now it'll be referred to simply as incomplete beta)
$$I(x; a,b) = \frac{B(x; a,b)}{B(a,b)}$$


## Debugging of [math-functions#35](https://github.com/bos/math-functions/issues/35)

Originally uncovered when plotting roundtrip error of `cumulative . quantile` for beta distribution in log scale.

In [ ]:
let fun x = let p  = invIncompleteBeta 7 0.07 x
                x' = incompleteBeta    7 0.07 p
            in x'
toRenderable
  $ plotFunctionsLog [\x -> logBase 10 $ relativeError (fun x) x] (1e-10, 1)

Oh! It's looks bad let plot how does incomplete beta looks like.

In [ ]:
-- Bird's eye view
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (1e-7,1e-6)
-- Closeup
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (2.8e-7,3e-7)

It's strange dip in smooth and _monotonic_ function! After adding couple of `traceShow`'s to incompleteBeta implementation it becomes clear that problem is lack of convergence. Initial guess is bad and 10 iterations is not enough. So we need to modify `invIncompleteBeta` for easy of debugging. We add  pluggable oracle for initial guess and return number of iterations together with result

In [ ]:
-- | Data type to describe approximation used. See 
data Guess
  = AS109
  | AS64_2
  | NR1
  | NR2
  deriving (Show,Eq)

data Res = Res
  { result :: !Double
  , nIter  :: !Int
  , iters  :: [Double]
  }
  deriving Show 

-- | Compute inverse of regularized incomplete beta function. Uses
-- initial approximation from AS109, AS64 and Halley method to solve
-- equation.
invIncompleteBeta' 
  :: (Double -> Double -> Double -> (Guess,Double))
  -> Double     -- ^ /p/ > 0
  -> Double     -- ^ /q/ > 0
  -> Double     -- ^ /a/ ∈ [0,0.5] !!!
  -> Res
invIncompleteBeta' oracle p q a
  | p <= 0 || q <= 0 =
      error $ printf "invIncompleteBeta p <= 0 || q <= 0.  p=%g q=%g a=%g" p q a
  | a <  0 || a >  1 =
      error $ printf "invIncompleteBeta x must be in [0,1].  p=%g q=%g a=%g" p q a
  | a == 0 || a == 1 = Res a 0 []
  | a > 0.5          = error "not implemented"
  | otherwise        = invIncompleteBetaWorker oracle (logBeta p q) p q  a


invIncompleteBetaWorker
  :: (Double -> Double -> Double -> (Guess,Double))
  -> Double -> Double -> Double -> Double -> Res
-- NOTE: p <= 0.5.
invIncompleteBetaWorker oracle beta a b p = loop (0::Int) [] (snd $ oracle a b p)
  where
    a1 = a - 1
    b1 = b - 1
    -- Solve equation using Halley method
    done i xs x = Res { result = x 
                      , nIter  = i
                      , iters  = reverse $ x : xs
                      }
    loop !i !xs !x
      | trace ("==== " ++ show i)
        $ traceShow (ulpDistance y p, y, p, y')
        $ traceShow (x, x')
        $ False = undefined
      -- We cannot continue at this point so we simply return `x'
      | x == 0 || x == 1             = done i xs x
      -- When derivative becomes infinite we cannot continue
      -- iterations. It can only happen in vicinity of 0 or 1. It's
      -- hardly possible to get good answer in such circumstances but
      -- `x' is already reasonable.
      | isInfinite f'                = done i xs x
      -- Iterations limit reached. Most of the time solution will
      -- converge to answer because of discreteness of Double. But
      -- solution have good precision already.
      | i >= 100                     = done i xs x
      -- Solution converges      
      | abs dx <= 16 * m_epsilon * x = done i xs x'
      -- OK we bumped into finite precision of floating point and 
      | within 2 y y' = done i xs x'
      | otherwise                    = loop (i+1) (x:xs) x'
      where
        y  = incompleteBeta_ beta a b x
        y' = incompleteBeta_ beta a b x'
        -- Calculate Halley step.
        f     = incompleteBeta_ beta a b x  - p
        f'  = exp $ a1 * log x + b1 * log1p (-x) - beta
        u   = f / f'
        dx  = u / (1 - 0.5 * min 1 (u * (a1 / x - b1 / (1 - x))))
        -- Next approximation. If Halley step leads us out of [0,1]
        -- range we revert to bisection.
        x'  | z < 0     = x / 2
            | z > 1     = (x + 1) / 2
            | otherwise = z
            where z = x - dx
            
-- Calculate initial guess. Approximations from AS64, AS109 and
-- Numerical recipes are used.
--
-- Equations are referred to by name of paper and number e.g. [AS64 2]
-- In AS64 papers equations are not numbered so they are refered
-- to by number of appearance starting from definition of
-- incomplete beta.
guessIIBeta :: Double -> Double -> Double -> (Guess,Double)
guessIIBeta a b p
      -- In this region we use approximation from AS109 (Carter
      -- approximation). It's reasonably good (2 iterations on
      -- average)
      | a > 1 && b > 1 =
          let r = (y*y - 3) / 6
              s = 1 / (2*a - 1)
              t = 1 / (2*b - 1)
              h = 2 / (s + t)
              w = y * sqrt(h + r) / h - (t - s) * (r + 5/6 - 2 / (3 * h))
          in (AS109, a / (a + b * exp(2 * w)))
      -- Otherwise we revert to approximation from AS64 derived from
      -- [AS64 2] when it's applicable.
      --
      -- It slightly reduces average number of iterations when `a' and
      -- `b' have different magnitudes.
      | chi2 > 0 && ratio > 1 = (AS64_2, 1 - 2 / (ratio + 1))
      -- If all else fails we use approximation from "Numerical
      -- Recipes". It's very similar to approximations [AS64 4,5] but
      -- it never goes out of [0,1] interval.
      | otherwise = case () of
          _| p < t / w  -> (NR1, (a * p * w) ** (1/a))
           | otherwise  -> (NR2, 1 - (b * (1 - p) * w) ** (1/b))
           where
             lna = log $ a / (a+b)
             lnb = log $ b / (a+b)
             t   = exp( a * lna ) / a
             u   = exp( b * lnb ) / b
             w   = t + u
      where
        -- Formula [2]
        ratio = (4*a + 2*b - 2) / chi2
        -- Quantile of chi-squared distribution. Formula [3].
        chi2 = 2 * b * (1 - t + y * sqrt t) ** 3
          where
            t   = 1 / (9 * b)
        -- `y' is Hasting's approximation of p'th quantile of standard
        -- normal distribution.
        y   = r - ( 2.30753 + 0.27061 * r )
                  / ( 1.0 + ( 0.99229 + 0.04481 * r ) * r )
          where
            r = sqrt $ - 2 * log p

incompleteBetaDeriv :: Double -> Double -> Double -> Double
incompleteBetaDeriv a b p
  = p**(a-1) * (1-p)**(b-1)

--invIncompleteBeta' guessIIBeta 7 0.07 2.8e-7
--invIncompleteBeta' guessIIBeta 7 0.07 2.88e-7

# Checking convergence

Pretty bad. Let plot number of iterations:

In [ ]:
toRenderable
  $ plotFunctions [fromIntegral . nIter . invIncompleteBeta' guessIIBeta 7 0.07
                  , const 10] (1e-7,1e-6)

That's pretty bad. Initial guess is very very poor.

So it's initial guess from AS64 fails us.

In [ ]:
nIterations =
  [ ((a-d,b-d),(a+d,b+d), fromIntegral (maximum iters) :: Double)
  | i <- [0 .. n-1]
  , j <- [0 .. n-1]
  , let a = d + fromIntegral i / n'
        b = d + fromIntegral j / n'
        -- Calculate number of iterations
        iters = parMap rpar (nIter . invIncompleteBeta' guessIIBeta a b) $ linspace (0,0.5) 100
  ]
  where
    n  = 20 :: Int
    n' = fromIntegral n :: Double
    d  = 1 / (2 * n')
    
toRenderable $ 
  layout_plots .~ 
    [ toPlot $ heat_map_values .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def

maximum [z | (_,_,z) <- nIterations]

In [ ]:
toRenderable
  $ plotFunctionsLog [logBase 10 . fromIntegral . nIter . invIncompleteBeta' guessIIBeta 0.5 0.5
                     , const (logBase 10 10)
                     ] (1e-10, 0.1)

In [ ]:
{-
let a = 0.5
    b = 0.5
 in toRenderable
  $ plotFunctionsLog
      [logBase 10 . fromIntegral . fst . invIncompleteBeta' guessIIBeta a b
      , const (logBase 10 10)
      ] (1e-10, 0.1)
-}
a = 0.5
b = 0.5

toRenderable
  $ plotFunctionsLog [ logBase 10 . fromIntegral . nIter . invIncompleteBeta' guessIIBeta a b
                     , const (logBase 10 10)
                     ] (1e-10, 0.45)

toRenderable
  $ plotFunctionsLog
      [ logBase 10 . result . invIncompleteBeta' guessIIBeta a b
      , logBase 10 . snd . guessIIBeta a b
      ] (1e-10, 0.45)
      
toRenderable
  $ plotFunctionsLog
      [ \p -> id $ snd (guessIIBeta a b p)
                         / result (invIncompleteBeta' guessIIBeta a b p)
      ] (1e-10, 0.45)

It looks like algorithm never converges in sense iterations never stop but it still give reasonable answer. We ned to investigae this

In [ ]:
(p,r) = head $ filter ((>90) . nIter. snd) 
 [ (p, invIncompleteBeta' guessIIBeta 0.5 0.5 p)
 | p <- logspace (1e-6,0.45) 1000
 ]
putStrLn $ "N_iter = " ++ show (nIter r) 
mapM_ print $ take 10 $ [0..] `zip` iters r


x1 = iters r !! 4
x2 = iters r !! 5
putStrLn $ "p  = " ++ show p
putStrLn $ "x1 = " ++ show x1
putStrLn $ "x2 = " ++ show x2
putStrLn $ "dx = " ++ show (ulpDistance x1 x2)
putStrLn $ "dp = " ++ show (ulpDistance (incompleteBeta 0.5 0.5 x1) (incompleteBeta 0.5 0.5  x2))

pts = [ (fromIntegral i,incompleteBeta 0.5 0.5 (addUlps (fromIntegral i) (min x1 x2)))
      | i <- [0 .. ulpDistance x1 x2]]

toRenderable
  $ layout_plots .~ 
      [ toPlot $ plot_lines_values .~ [pts] $ def]
  $ def

In [ ]:
toRenderable
  $ plotFunctions
      [ incompleteBeta 0.5 0.5
      ] (0.5e-10, 2e-10)
toRenderable
  $ plotFunctions
      [ incompleteBetaDeriv 0.5 0.5
      ] (0.5e-10, 2e-10)

Derivative?

In [ ]:
pts = [ ( fromIntegral i :: Double
        , fromIntegral (ulpDistance y0 $ incompleteBeta 0.5 0.5 (addUlps (fromIntegral i) x0)) :: Double
        )
      | i <- [0 .. 200]]
  where
    x0 = 1e-10
    y0 = incompleteBeta 0.5 0.5 x0

toRenderable
  $ layout_plots .~ 
      [ toPlot $ plot_lines_values .~ [pts] $ def]
  $ def 
